In [1]:
import datetime
import pyaurorax

aurorax = pyaurorax.PyAuroraX()
at = aurorax.tools

# Create SMILE ASI movie

There is a handy function available that makes a movie from any list of image files. This means that we need to generate the files first, and then pass the filenames into the function. 

Let's have a look at an example of making a movie for an hour of SMILE ASI data.

In [2]:
# download an hour of SMILE ASI data
dataset_name = "SMILE_ASI_RAW"
start_dt = datetime.datetime(2025, 1, 1, 9, 0)
end_dt =datetime.datetime(2025, 1, 1, 9, 59)
site_uid = "luck"
r = aurorax.data.ucalgary.download(dataset_name, start_dt, end_dt, site_uid=site_uid)

In [3]:
# read in the hour of data
data = aurorax.data.ucalgary.read(r.dataset, r.filenames, n_parallel=5)

In [4]:
# scale all the images
#
# NOTE: you can scale all images or just one image
images_scaled = at.scale_intensity(data.data, min=10, max=70)

To prep the frames for the movie function, we'll use the `display()` function and have it return the figure, then we'll add some timestamp and other text to our liking.

Since there's a lot of frames and this task is very parallel-friendly, we'll leverage Python's `multiprocessing` and `tqdm` libraries to help us speed things up.

In [5]:
import os
import platform
import matplotlib.pyplot as plt
from tqdm.contrib.concurrent import process_map as tqdm_process_map
from tqdm import tqdm


def process_frame(i):
    _, ax = at.display(images_scaled[:, :, :, i], returnfig=True)
    ax.text(5, 450, "SMILE ASI", color="white", size=16)
    ax.text(5, 420, f"{site_uid.upper()}", color="white", size=16)
    ax.text(350, 8, data.timestamp[i].strftime("%Y-%m-%d %H:%M:%S UTC"), color="white", size=11)
    filename = "movie_frames/%s_%s_luck_smile_asi.png" % (data.timestamp[i].strftime("%Y%m%d_%H%M%S"), site_uid)
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    plt.savefig(filename, bbox_inches="tight")
    plt.close()
    return filename


if (platform.system() == "Windows"):
    # pre-process frames serially
    #
    # NOTE: multiprocessing on Windows from within a notebook is not too easy, so we'll
    # just do this serially.
    frame_filename_list = []
    for i in tqdm(range(0, images_scaled.shape[-1]), total=images_scaled.shape[-1], desc="Generating frame files: ", unit="frames"):
        frame_filename_list.append(process_frame(i))
else:
    frame_filename_list = tqdm_process_map(
        process_frame,
        range(0, images_scaled.shape[-1]),
        max_workers=5,
        chunksize=1,
        desc="Generating frame files: ",
        unit="frames",
    )

Generating frame files:   0%|          | 0/1200 [00:00<?, ?frames/s]

Now that we have our frames saved to disk, we'll use the `movie()` function to create the movie.

In [6]:
# generate the movie
at.movie(frame_filename_list, "test.mp4", n_parallel=5)

Reading files:   0%|          | 0/1200 [00:00<?, ?files/s]

Encoding frames:   0%|          | 0/1200 [00:00<?, ?frames/s]